In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import os
import logging
import cx_Oracle
import time
import json
from datetime import date, timedelta

In [2]:
def get_previous_business_day(today_date=None):
    """
    Returns the previous business day (Mon-Fri).
    On Monday, returns Friday. Skips weekends automatically.
    """
    if today_date is None:
        today_date = date.today()
    
    day_offset = 1
    while True:
        previous_day = today_date - timedelta(days=day_offset)
        if previous_day.weekday() < 5:  # 0-4 = Mon-Fri
            return previous_day
        day_offset += 1

previous_bd = get_previous_business_day()
previous_bd = previous_bd.strftime('%Y-%m-%d')

In [1]:
output_dir='/home/hyungshin.lee/dbqm_data/input'

start = time.time()
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Database connection details
host = '192.168.1.27'
port = '3306'
db_name = 'quantdb_maria'
username = 'quantdb'
password = 'QuantDb2023!'

# Create engine for database connection
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{db_name}")

# with open('/home/hyungshin.lee/dbqm/app/config.json', 'r', encoding='utf-8') as f:
#     config = json.load(f)

period1= '2017-01-01'
period2= previous_bd

# Function to execute a query and return a DataFrame
def query(sql):
    with engine.connect() as conn:
        temp_query = text(sql)
        return pd.read_sql(temp_query, conn)

# Function to get raw tables
def get_raw_tables():
    #tables = ['COM_Table', 'CON_Table', 'NFS_IFRS_Table', 'SSC', 'CIA_Table', 'FS1_Table']
    #tables = ['fn_COM', 'fn_CON', 'NFS_IFRS_Table', 'fn_NFR_IFRS', 'fn_SSC', 'fn_CIA', 'fn_FS1']
    tables = ['fn_COM', 'fn_CON', 'fn_NFS_IFRS', 'fn_NFR_IFRS', 'fn_SSC', 'fn_CIA', 'fn_FS1']
    raw_tables = {table: query(f"SELECT * FROM {table}") for table in tables}
    
    for table in tables:
        logging.info(f'{table} Done')
    
    return raw_tables

# Function to get all KOSPI200 symbols
def get_all_kospi200():
    sql = "SELECT UNIQUE(Symbol) from fn_COM where KOSPI200YN = 'Y'"
    return query(sql)

# Fetch raw tables
datatable = get_raw_tables()

# Creating a unified column dictionary for tables
cols_dict = {col: table for table, df in datatable.items() for col in df.columns}

# List of columns to be saved
columns_to_save = [
    'AdjPrc', 'AdjPrc_High_60D', 'MktCap', 'ListedShares', 'Sales_TQ_ic', 'Sales_LQ_ic', 'Sales_2LQ_ic',
    'Sales_3LQ_ic', 'OI_TQ_ic', 'OI_LQ_ic', 'OI_2LQ_ic', 'OI_3LQ_ic', 'CIE_TQ_ic', 'CIE_LQ_ic', 'CIE_2LQ_ic',
    'CIE_3LQ_ic', 'NICI_TQ_ic', 'NICI_LQ_ic', 'NICI_2LQ_ic', 'NICI_3LQ_ic', 'OI_E3_ic', 'OI_E3_NextYear_ic', 'OI_E3_2yr_ic', 
    'NIP_E3_ic', 'NIP_E3_NextYear_ic', 'KLCAIndustry', 'NetVol_Inst_20D', 'Dep_Amort_TQ_fs1', 'Dep_Amort_LQ_fs1',
    'Dep_Amort_2LQ_fs1', 'Dep_Amort_3LQ_fs1', 'Int_Inc_TQ_fs1', 'Int_Inc_LQ_fs1', 'Int_Inc_2LQ_fs1', 'Int_Inc_3LQ_fs1',
    'Int_Exp_TQ_fs1', 'Int_Exp_LQ_fs1', 'Int_Exp_2LQ_fs1', 'Int_Exp_3LQ_fs1', 'Div_fs1', 'Assets_TQ_fs1', 'Assets_LQ_fs1',
    'Assets_2LQ_fs1', 'Assets_3LQ_fs1', 'TA_TQ_fs1', 'TA_LQ_fs1', 'TA_2LQ_fs1', 'TA_3LQ_fs1', 'Amt_Avg_20D', 'Num_EPS_ESts_E3',
    'FG_IndustryGroup_Idx','KSECapSize'
]

# Create the directory if it doesn't exist
os.makedirs(output_dir+'/dataFinal/', exist_ok=True)

2025-04-29 18:05:59,495 - INFO - fn_COM Done
2025-04-29 18:05:59,498 - INFO - fn_CON Done
2025-04-29 18:05:59,498 - INFO - fn_NFS_IFRS Done
2025-04-29 18:05:59,499 - INFO - fn_NFR_IFRS Done
2025-04-29 18:05:59,500 - INFO - fn_SSC Done
2025-04-29 18:05:59,500 - INFO - fn_CIA Done
2025-04-29 18:05:59,501 - INFO - fn_FS1 Done


In [3]:
stringSave = ['KLCAIndustry','KSECapSize', 'FG_IndustryGroup_Idx']

for column in columns_to_save:
    table_name = cols_dict.get(column)
    df = datatable[table_name]
    filename = output_dir+f'/dataFinal/{column}.pkl'
    
    # Convert column to numeric if it's not in the string_save list
    if column not in stringSave:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    
    # Select relevant columns and save as pickle
    df_to_save = df[['Dates', 'Symbol', 'Name', column]].drop_duplicates()
    df_to_save.to_pickle(filename)
    logging.info(f'{filename} saved')
end = time.time()
print(f'{end - start} time consumed')

2025-04-30 14:05:37,721 - INFO - /home/hyungshin.lee/dbqm_data/input/dataFinal/AdjPrc.pkl saved
2025-04-30 14:05:42,861 - INFO - /home/hyungshin.lee/dbqm_data/input/dataFinal/AdjPrc_High_60D.pkl saved
2025-04-30 14:05:47,559 - INFO - /home/hyungshin.lee/dbqm_data/input/dataFinal/MktCap.pkl saved
2025-04-30 14:05:52,397 - INFO - /home/hyungshin.lee/dbqm_data/input/dataFinal/ListedShares.pkl saved
2025-04-30 14:06:01,447 - INFO - /home/hyungshin.lee/dbqm_data/input/dataFinal/Sales_TQ_ic.pkl saved
2025-04-30 14:06:08,654 - INFO - /home/hyungshin.lee/dbqm_data/input/dataFinal/Sales_LQ_ic.pkl saved
2025-04-30 14:06:16,169 - INFO - /home/hyungshin.lee/dbqm_data/input/dataFinal/Sales_2LQ_ic.pkl saved
2025-04-30 14:06:23,489 - INFO - /home/hyungshin.lee/dbqm_data/input/dataFinal/Sales_3LQ_ic.pkl saved
2025-04-30 14:06:31,123 - INFO - /home/hyungshin.lee/dbqm_data/input/dataFinal/OI_TQ_ic.pkl saved
2025-04-30 14:06:39,068 - INFO - /home/hyungshin.lee/dbqm_data/input/dataFinal/OI_LQ_ic.pkl saved

73521.3662724495 time consumed


### 저장되어 있는 구조 보기

In [4]:
# df1 = pd.read_pickle(output_dir+'/dataFinal/AdjPrc.pkl') # 수정주가

In [5]:
# df1.tail()

,Dates,Symbol,Name,AdjPrc
4509467,2025-04-23,A900140,엘브이엠씨홀딩스,1746.0
4509468,2025-04-23,A950010,평산차업 KDR,NaN
4509469,2025-04-23,A950070,중국고섬,NaN
4509470,2025-04-23,A950100,SBI모기지,NaN
4509471,2025-04-23,A950210,프레스티지바이오파마,15200.0


In [6]:
# df2 = pd.read_pickle(output_dir+'/dataFinal/MktCap.pkl') # 시가총액

In [7]:
# df2.tail()

,Dates,Symbol,Name,MktCap
4509467,2025-04-23,A900140,엘브이엠씨홀딩스,299593.0
4509468,2025-04-23,A950010,평산차업 KDR,NaN
4509469,2025-04-23,A950070,중국고섬,NaN
4509470,2025-04-23,A950100,SBI모기지,NaN
4509471,2025-04-23,A950210,프레스티지바이오파마,913462.0


### 지수비중 데이터

In [8]:
dsn = cx_Oracle.makedsn('192.168.1.5', '1521', service_name = 'ORA11')
con = cx_Oracle.connect('dbam', 'dbamkass', dsn = dsn, encoding="UTF-8")
cur = con.cursor()

sql_k200 = " select 일자, 종목코드2 종목코드, 종목명_국문 종목명, 당일가격, 지수내비중, 상장주식수, 유동비율, \
            지수주식수, 지수시가총액 \
             from KASS1.TH068 \
             where 익일적용여부 ='NXT' \
             and ISIN = 'KRD020020016' \
             order by 지수내비중 desc "

cur.execute(sql_k200)
indexCap = pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description])

indexCap['일자']=pd.to_datetime(indexCap['일자'], format ='%Y%m%d')
indexCap.set_index(['일자', '종목코드'], inplace=True)
indexCap.index.names=['Dates', 'Symbol']
indexCap.columns=['name', 'close', 'idx_weight', 'num_shares', 'float_rate', 'idx_num_shares', 'idx_cap']
# indexCap.to_pickle('./data/all_data2407temp/indexCapTotal.pkl')

In [9]:
indexCap.to_pickle(output_dir+'/dataFinal/indexCapTotal.pkl')

In [10]:
indexCap.tail()

name  close  idx_weight  num_shares  float_rate  \
Dates      Symbol                                                            
2016-07-12 A000140  하이트진로홀딩스보통주  14000        0.01    23206765         0.3   
           A004700      조광피혁보통주  43650        0.01     6649138         0.3   
2016-07-13 A000140  하이트진로홀딩스보통주  13850        0.01    23206765         0.3   
           A004700      조광피혁보통주  43950        0.01     6649138         0.3   
2016-07-14 A000140  하이트진로홀딩스보통주  13700        0.01    23206765         0.3   

                    idx_num_shares      idx_cap  
Dates      Symbol                                
2016-07-12 A000140        23206765  97468413000  
           A004700         6649138  87070462110  
2016-07-13 A000140        23206765  96424108575  
           A004700         6649138  87668884530  
2016-07-14 A000140        23206765  95379804150

### ESG 데이터

In [11]:
# esg_raw = pd.read_excel('esg to data.xlsx')
# esg_raw.iloc[0,].fillna(method='ffill',inplace=True)
# col_list = ['종목명', 'ASymbol', '등급', '등급_전체']
# cols = []
# for col in esg_raw.columns:
#     for tocol in col_list:
#         if  esg_raw[col].isin([tocol]).any():
#             cols.append(col)
# esg_raw = esg_raw[cols]
# # esg_raw = esg_raw.drop(0)
# esg_raw = esg_raw.drop([0,1])
# esg_raw.columns = ['Name', 'Symbol'] + esg_raw.columns[2:].tolist()
# esg_raw = esg_raw.set_index(['Symbol','Name'])
# cols = []
# for col in esg_raw.columns:
#     col = str(col)
#     col_tmp = col.replace('_','')[2:]
#     col_tmp1 = col_tmp[2:4]
#     col_tmp2 = col_tmp[0:2]
#     col_tmp = col_tmp1 + col_tmp2
#     cols.append(col_tmp)        
# esg_raw.columns = cols
# # create a excel writer object
# with pd.ExcelWriter("./dataFinal/esg_temp.xlsx") as writer:
#     for col in esg_raw.columns:
#         esg_raw[col].to_excel(writer, sheet_name=col)